# Assignment 1: Dynamic Programming

---

## Task 1) Edit Distances

Implement the [Hamming](https://en.wikipedia.org/wiki/Hamming_distance) and [Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) (edit) distances and compute them for the for the following word pairs.
It may help to compute them by hand first.

<img src = "./assets/97090.jpg" width="33%" /> <img src = "./assets/97222.jpg" width="33%" /> <img src = "./assets/97669.jpg" width="33%" />

Aside from computing the distance (which is a scalar), do the backtrace and compute the edit transcript (and thus alignment).

---

In [1]:
WORD_PAIRS = [
    ("GCGTATGAGGCTAACGC", "GCTATGCGGCTATACGC"),
    ("kühler schrank", "schüler krank"),
    ("the longest", "longest day"),
    ("nicht ausgeloggt", "licht ausgenockt"),
    ("gurken schaben", "schurkengaben")
]

In [2]:
def hamming(s1: str, s2: str) -> int:
    """
    Compute the hamming distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int.
    """
    # Hint: Think about how you can deal with unequal word lengths.
    
    ### YOUR CODE HERE
    dist = 0

    max_len = max(len(s1), len(s2))
    s1 = s1.ljust(max_len); s2 = s2.ljust(max_len)

    for c1, c2 in zip(s1, s2):
        dist += 1 if c1 != c2 else 0

    return dist
    ### END YOUR CODE

In [3]:
for wordpair in WORD_PAIRS:
    dist = hamming(s1=wordpair[0], s2=wordpair[1])
    print("hamming('{}', '{}') = {}".format(
        wordpair[0], wordpair[1], dist
    ))

### EXPECTED
# hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
# hamming('kühler schrank', 'schüler krank') = 13
# hamming('the longest', 'longest day') = 11
# hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
# hamming('gurken schaben', 'schurkengaben') = 14

hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
hamming('kühler schrank', 'schüler krank') = 13
hamming('the longest', 'longest day') = 11
hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
hamming('gurken schaben', 'schurkengaben') = 14


In [4]:
def levenshtein(s1: str, s2: str, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}) -> (int, str):
    """
    Compute the levenshtein (edit) distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int and edit transcript as string.
    """
    # Hint: The probably most intuitive approach is bottom-up.
    
    ### YOUR CODE HERE
    import numpy as np
    D = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)
    D[0, 1:] = range(1, len(s2) + 1)
    D[1:, 0] = range(1, len(s1) + 1)


    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            delta = cost["m"] if s1[i-1] == s2[j-1] else cost["s"]
            D[i, j] = min(
                D[i-1, j] + cost["d"],
                D[i, j-1] + cost["i"],
                D[i-1, j-1] + delta
            )

    i, j = len(s1), len(s2)
    edits = ""

    while i > 0 and j > 0:
        min_idx = np.argmin((
            D[i-1, j] + cost["d"],
            D[i, j-1] + cost["i"],
            D[i-1, j-1] + (cost["m"] if s1[i-1] == s2[j-1] else cost["s"])
        ))

        match min_idx:
            case 0:
                edits += "d"
                i -= 1
            case 1:
                edits += "i"
                j -= 1
            case 2:
                edits += "m" if s1[i-1] == s2[j-1] else "s"
                i -= 1; j -= 1

    edits += "".join(["d" for _ in range(i)])
    edits += "".join(["i" for _ in range(j)])

    return D[len(s1), len(s2)], edits[::-1]
    ### END YOUR CODE

In [5]:
for wordpair in WORD_PAIRS:
    dist, transcript = levenshtein(s1=wordpair[0], s2=wordpair[1])
    print("levenshtein('{}', '{}') = {} ({})".format(
        wordpair[0], wordpair[1], dist, transcript
    ))

### EXPECTED
# levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
# levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
# levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
# levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
# levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)

levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)


---

## Task 2) Basic Spelling Correction using Edit Distance

For spelling correction, we will use prior knowledge, to put _some_ learning into our system.

The underlying idea is the _Noisy Channel Model_, that is: The user _intends_ to write a word `w`, but through some noise in the process, happens to type the word `x`.

The correct word $\hat{w}$ is that word, that is a valid candidate and has the highest probability:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} P(w | x) \\
        & = & \argmax_{w \in V} \frac{P(x|w) P(w)}{P(x)} \\
        & = & \argmax_{w \in V} P(x|w) P(w)
\end{eqnarray}
$$

1. The candidates $V$ can be obtained from a _vocabulary_.
2. The probability $P(w)$ of a word $w$ can be _learned (counted) from data_.
3. The probability $P(x|w)$ is more complicated... It could be learned from data, but we could also use a _heuristic_ that relates to the edit distance, e.g. rank by distance.

You may use [Peter Norvig's count_1w.txt](http://norvig.com/ngrams/) file, [local mirror](res/count_1w.tar.bz2).
Note that it may help to restrict to the first 10k words to get started.

---

In [6]:
EXAMPLES = [
    "pirates",    # in-voc
    "pirutes",    # pirates?
    "continoisly",  # continuosly?
]

In [7]:
### TODO: Prepare the vocabulary

### YOUR CODE HERE
word_file = "C:\\Users\\Felix\\PythonProjects\\seqlrn_assignments\\1-dynamic-programming\\data\\count_1w\\count_1w.txt"

with open(word_file) as wf:
    lines = wf.readlines()#[:10_000]
    V = {
        l.split("\t")[0]: int(l.rstrip("\n").split("\t")[1]) for l in lines
    }

sum_counts = sum(V.values())
P = {
    word: count / sum_counts for word, count in V.items()
}

print(V["the"])
print(P["the"])
### END YOUR CODE

23135851162
0.03933837507090547


In [8]:
def suggest(w: str, dist_fn, max_cand=5) -> list:
    """
    Compute suggestions for spelling correction using edit distance.
    
    Arguments:
    w: Word in question.
    dist_fn: Distance function to use (e.g. levenshtein).
    max_cand: Maximum number of suggestions.

    Returns a list of tuples (word, dist, score) sorted by score and distance.
    """
    ### YOUR CODE HERE
    canditates = []

    for entry in V.keys():
        score = P[entry]  #  i didnt understand what the score should be so I took the pure probability of vocabulary entry 
        dist, _ = dist_fn(w, entry)

        canditates.append((entry, int(dist), score))

    canditates.sort(key=lambda c: (c[1], -c[2]))
    
    return canditates[:max_cand]
    ### END YOUR CODE

In [9]:
# How does your suggest function behave with levenshtein distance?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=levenshtein, max_cand=3)
    print("{} {}".format(word, suggestions))

### EXPECTED
### Notice: your scores may vary!
# pirates [('pirates', 0, -11.408058827802126)]
# pirutes [('pirates', 1, -11.408058827802126), ('minutes', 2, -8.717825438953103), ('viruses', 2, -11.111468702571859)]
# continoisly [('continously', 1, -15.735337826575178), ('continuously', 2, -11.560071979871001), ('continuosly', 2, -17.009283000138204)]

pirates [('pirates', 0, 1.1105624927202025e-05), ('pirate', 1, 7.325731286206999e-06), ('pilates', 1, 4.107259856143899e-06)]
pirutes [('pirates', 1, 1.1105624927202025e-05), ('minutes', 2, 0.0001636426552359956), ('viruses', 2, 1.4939995154775671e-05)]
continoisly [('continously', 1, 1.4663228794178848e-07), ('continuously', 2, 9.539476198099983e-06), ('continuosly', 2, 4.1016845033741084e-08)]


### Food for Thought

- How would you modify the implementation so that it works fast for large lexica (eg. the full file above)?
- How would you modify the implementation so that it works "while typing" instead of at the end of a word?
- How do you handle unknown/new words?

---

## Task 3) Needleman-Wunsch: Keyboard-aware Auto-Correct

In the parts 1 and 2 above, we applied uniform cost to all substitutions.
This does not really make sense if you look at a keyboard: the QWERTY layout will favor certain substitutions (eg. _a_ and _s_), while others are fairly unlikely (eg. _a_ and _k_).

Implement the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm) which is very similar to the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance), however it doesn't _minimize the cost_ but _maximizes the similarity_.
For a good measure of similarity, implement a metric that computes a meaningful weight for a given character substitution.

---

In [10]:
def keyboardsim(s1: str, s2: str) -> float:
    ### YOUR CODE HERE
    keyboard_layout = {
        'q': (0, 0), 'w': (0, 1), 'e': (0, 2), 'r': (0, 3), 't': (0, 4), 'y': (0, 5), 'u': (0, 6), 'i': (0, 7), 'o': (0, 8), 'p': (0, 9),
        'a': (1, 0), 's': (1, 1), 'd': (1, 2), 'f': (1, 3), 'g': (1, 4), 'h': (1, 5), 'j': (1, 6), 'k': (1, 7), 'l': (1, 8),
        'z': (2, 0), 'x': (2, 1), 'c': (2, 2), 'v': (2, 3), 'b': (2, 4), 'n': (2, 5), 'm': (2, 6)
    }
    
    x1, y1 = keyboard_layout[s1]
    x2, y2 = keyboard_layout[s2]
    
    keyboard_dist = abs(x1 - x2) + abs(y1 - y2)  # Manhattan distance
    
    return 1 / (1 + keyboard_dist)
    ### END YOUR CODE


def nw(s1: str, s2: str, d: float, sim_fn) -> float:
    """
    Apply needleman-wunsch algorithm.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    d: Gap penalty score.
    sim_fn: Similarity function to use.

    Returns the score as float.
    """
    ### YOUR CODE HERE
    import numpy as np
    D = np.zeros((len(s1) + 1, len(s2) + 1), dtype=int)
    D[0, 1:] = range(1, len(s2) + 1); D[0, 1:] *= d
    D[1:, 0] = range(1, len(s1) + 1); D[1:, 0] *= d

    for i in range(1, len(s1) + 1):
        for j in range(1, len(s2) + 1):
            cs = D[i-1, j-1] + sim_fn(s1[i-1], s2[j-1])
            cd = D[i-1, j] + d
            ci = D[i, j-1] + d
            D[i,j] = max(cs, cd, ci) 
    
    return D[len(s1), len(s2)]
    ### END YOUR CODE


def nw_based_dist(s1: str, s2: str) -> (int, str):
    """
    Compute the needleman-wunsch distance between two strings.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    
    Returns the distance as int and <unsupported> string.
    """
    ### YOUR CODE HERE
    return -nw(s1, s2, -1, keyboardsim), "<unsupported>"    
    ### END YOUR CODE

In [11]:
# How does your suggest function behave with nw and a keyboard-aware similarity?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=nw_based_dist, max_cand=3)
    print("{} {}".format(word, suggestions))

pirates [('pirates', -7, 1.1105624927202025e-05), ('emirates', -6, 1.221171268497735e-05), ('pilates', -6, 4.107259856143899e-06)]
pirutes [('pirates', -6, 1.1105624927202025e-05), ('minutes', -5, 0.0001636426552359956), ('viruses', -5, 1.4939995154775671e-05)]
continoisly [('continously', -10, 1.4663228794178848e-07), ('continuously', -9, 9.539476198099983e-06), ('continuosly', -9, 4.1016845033741084e-08)]


### Food for Thought

- What could be better heuristics for similarity resp. cost of substitution than the one above?
- What about capitalization, punctiation and special characters?
- What about swipe-to-type?